In [1]:
import pandas as pd
import requests
import json
import re
import sys
import numpy as np
import os
import nltk
import pickle
from sklearn.utils import shuffle

sys.path.append('../src')

from commons import elastic
from langdetect import detect
from text_processing import text_normalizer
from importlib import reload
from text_processing import duplicate_finder
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import spacy
text_normalizer = reload(text_normalizer)
from dateutil.parser import parse
from utilities import excel_writer, excel_reader
from text_processing import abbreviations_resolver
abbreviations_resolver = reload(abbreviations_resolver)
from utilities import utils

stemmer = SnowballStemmer("english")
lmtzr = WordNetLemmatizer()
nlp = spacy.load('en_core_web_sm')

#nlp = spacy.load("en_core_web_md")

filter_word_list = text_normalizer.build_filter_dictionary(["../data/Filter_Geo_Names.xlsx"])

_abbreviations_resolver = abbreviations_resolver.AbbreviationsResolver(filter_word_list)
_abbreviations_resolver.load_model("../model/abbreviations_dicts")

C:\Users\Maryia_Ivanina\Anaconda3\envs\cornell-copy-new\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# check that the environment is ready and properly installed
from allennlp.predictors.predictor import Predictor
predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/coref-model-2018.02.05.tar.gz")
predictor.predict(
    document = """This paper considers the stochastic production frontier approach to analyse the technical efficiency and its determinants of fish farms in Ghana using a cross-section data of 150 farms. It considers the explicit effects of family and hired labour on production by setting the log-value of the zero-observation of these two sources of labour to zero with dummy variables. Results demonstrate that expected elasticities of mean output with respect to all input variables are positive and significant. Findings also show that family and hired labour used for fish farming in Ghana may be equally productive. Fish farms in Ghana are revealed to be characterised by technology with increasing return to scale. The combined effects of operational and farm specific factors are found to influence efficiency. The study further reveals that inclusion of interaction between some exogenous variables in the inefficiency model is significant in explaining the variation in efficiency. Results also suggest that small pond operators are more efficient than farms with large ponds. Mean technical efficiency is estimated to be 78 percent."""
)

_jsonnet not loaded, treating C:\Users\MARYIA~1\AppData\Local\Temp\tmpul1cme7r\config.json as json
C:\Users\Maryia_Ivanina\Anaconda3\envs\cornell-copy-new\lib\site-packages\torch\nn\modules\rnn.py:46: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
C:\Users\Maryia_Ivanina\Anaconda3\envs\cornell-copy-new\lib\site-packages\allennlp\data\token_indexers\token_characters_indexer.py:51: UserWarning: You are using the default value (0) of `min_padding_length`, which can cause some subtle bugs (more info see https://github.com/allenai/allennlp/issues/1954). Strongly recommend to set a value, usually the maximum size of the convolutional layer size when using CnnEncoder.
  UserWarning)


{'top_spans': [[0, 1],
  [3, 6],
  [3, 7],
  [3, 12],
  [9, 9],
  [10, 12],
  [14, 14],
  [14, 18],
  [14, 20],
  [20, 20],
  [22, 29],
  [28, 29],
  [31, 31],
  [32, 32],
  [33, 42],
  [37, 42],
  [39, 39],
  [40, 40],
  [40, 42],
  [42, 42],
  [44, 44],
  [50, 59],
  [55, 59],
  [59, 59],
  [61, 61],
  [66, 66],
  [70, 72],
  [70, 73],
  [72, 73],
  [77, 79],
  [85, 85],
  [88, 89],
  [89, 89],
  [89, 92],
  [89, 98],
  [91, 91],
  [92, 92],
  [92, 98],
  [93, 93],
  [95, 96],
  [98, 98],
  [104, 107],
  [107, 107],
  [109, 109],
  [114, 114],
  [117, 117],
  [119, 119],
  [121, 129],
  [125, 127],
  [125, 129],
  [127, 127],
  [131, 131],
  [133, 134],
  [134, 134],
  [136, 138],
  [139, 139],
  [143, 147],
  [145, 147],
  [145, 151],
  [149, 151],
  [153, 153],
  [155, 155],
  [156, 157],
  [156, 159],
  [159, 159],
  [161, 161],
  [163, 163],
  [165, 167],
  [166, 166],
  [169, 175],
  [172, 172],
  [174, 175],
  [177, 177],
  [177, 179]],
 'predicted_antecedents': [-1,
  -1,
  -1

# Checking content of fao data

In [1]:
from bs4 import BeautifulSoup
content = None 
with open(r"..\tmp\Example FAO data\XF0.xml", "rb") as f:
    content = BeautifulSoup(f.read(), "lxml")

In [7]:
content.find("arr", {"name": "title_eng"})

<arr name="title_eng"><str>Report on the 1980 World census of agriculture. Results by countries: Canada, Portugal</str></arr>

In [12]:
content.find("doc")

<doc><str name="URI">http://agris.fao.org/aos/records/XF8658403</str><arr name="fsURI"><str>1987/XF/XF1987_0.rdf</str></arr><str name="ARN">XF8658403</str><int name="date">1987</int><str name="center">http://aims.fao.org/node/122621</str><arr name="title_eng"><str>Report on the 1980 World census of agriculture. Results by countries: Canada, Portugal</str></arr><arr name="corporateAuthor"><str>FAO, Rome (Italy). Statistics Div.</str></arr><arr name="publicationDate"><str>1985</str></arr><arr name="subject"><str>canada</str><str>portugal</str><str>censuses/ farmland</str><str>farm area</str><str>livestock numbers</str><str>plant production</str><str>crops</str><str>canada</str><str>portugal</str><str>recensement/ surface agricole utile</str><str>surface d\'exploitation</str><str>effectif du cheptel</str><str>production vegetale</str><str>plante de culture</str><str>canada</str><str>portugal</str><str>censos/ tierras agricolas</str><str>area total de la explotacion</str><str>numero de cab

In [40]:
with open(r"..\tmp\Example FAO data\XF0.xml", "r") as f:
    print(f.read()[:1000])

b'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n<lst name="responseHeader"><int name="status">0</int><int name="QTime">144</int><lst name="params"><str name="q">center:"http://aims.fao.org/node/122621"</str><str name="rows">232042</str><str name="wt">xml</str></lst></lst><result name="response" numFound="232042" start="0"><doc><str name="URI">http://agris.fao.org/aos/records/XF8658403</str><arr name="fsURI"><str>1987/XF/XF1987_0.rdf</str></arr><str name="ARN">XF8658403</str><int name="date">1987</int><str name="center">http://aims.fao.org/node/122621</str><arr name="title_eng"><str>Report on the 1980 World census of agriculture. Results by countries: Canada, Portugal</str></arr><arr name="corporateAuthor"><str>FAO, Rome (Italy). Statistics Div.</str></arr><arr name="publicationDate"><str>1985</str></arr><arr name="subject"><str>canada</str><str>portugal</str><str>censuses/ farmland</str><str>farm area</str><str>livestock numbers</str><str>plant production</str><str>crops</str><st

To see what one document is about in the agris dao website, you can take the content of <str name="URI">http://agris.fao.org/aos/records/XF8658403</str> and use XF8658403 as a record Id to form this url https://agris.fao.org/agris-search/search.do?recordID=XF8658403

# Webscrapping agris website

In [38]:
from webscrapping import webcrawler_base
webcrawler_base = reload(webcrawler_base)
from webscrapping import webcrawler_agris
webcrawler_agris = reload(webcrawler_agris)
_webcrawler_agris = webcrawler_agris.WebCrawlerAGRIS()

In [18]:
_webcrawler_agris.crawl_query("https://agris.fao.org/agris-search/searchIndex.do?query=small+scale+farming+ghana&selectedLanguage=0&countryResource=0&selectedResourceType=Both&enableField=Disable&typeresultsField=Both&aggregatorField=Disable&fromDate=&toDate=&dataProvider=&fullTextField=Disable&agrovocString=&centerFilterList=&typeString=journal_article&sortField=Relevance&sortOrder=Descending", "../tmp/Agris_temp")



Page 0
Fetch https://agris.fao.org/agris-search/biblio.action?advQuery=small+scale+farming+ghana&selectedLanguage=0&countryResource=0&selectedResourceType=Both&enableField=Disable&typeresultsField=Both&aggregatorField=Disable&fromDate=&toDate=&dataProvider=&fullTextField=Disable&agrovocString=&centerFilterList=&typeString=journal_article&sortField=Relevance&sortOrder=Descending&startIndexSearch=0
Url was searched
search.do?recordID=DJ20210562064
Fetch http://agris.fao.org/agris-search/search.do?recordID=DJ20210562064 to ../tmp/Agris_temp\DJ20210562064.html
Url was searched
sleep 1s
search.do?recordID=QL2018001495
Fetch http://agris.fao.org/agris-search/search.do?recordID=QL2018001495 to ../tmp/Agris_temp\QL2018001495.html
Url was searched
sleep 1s
search.do?recordID=JP2020003516
Fetch http://agris.fao.org/agris-search/search.do?recordID=JP2020003516 to ../tmp/Agris_temp\JP2020003516.html
Url was searched
sleep 1s
search.do?recordID=JP2020003514
Fetch http://agris.fao.org/agris-search

In [39]:
_webcrawler_agris.prepare_dataset("../tmp/Agris_temp", "../tmp/AGRIS_temp.xlsx")

Saving...
Saved to ../tmp/AGRIS_temp.xlsx
